<a href="https://colab.research.google.com/github/talhaanwarch/EEG_mental_task/blob/master/rnn_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mne
!pip install wfdb

     |████████████████████████████████| 6.6MB 6.4MB/s 
     |████████████████████████████████| 102kB 5.9MB/s 
     |████████████████████████████████| 163kB 40.8MB/s 
  Created wheel for wfdb: filename=wfdb-2.2.1-cp36-none-any.whl size=100368 sha256=309e66a4a707ac04ac71310c11437b28a7116691eb68a67700cc66814bd27122
  Stored in directory: /root/.cache/pip/wheels/bb/a9/00/0078d26b0c15b31be0001af8eb659496709c361c69641303f1
Successfully built wfdb


In [2]:
import wfdb
dbs = wfdb.get_dbs()
wfdb.dl_database('eegmat','data')

Created local base download directory: data
Finished downloading files


In [0]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import progressbar
import re
import mne
from scipy import stats

In [4]:
import os
import glob
path = r'/content/data/' # use your path
all_files = glob.glob(os.path.join(path, "*.edf")) 

data1=[]
data2=[]
for filename in (all_files):
    if int(re.findall(r'\d+',filename)[1])==1:
        data1.append(mne.io.read_raw_edf(filename,preload=True).get_data()[0:-3,::]);
    else:
        data2.append(mne.io.read_raw_edf(filename,preload=True).get_data()[0:-3,::]);

Extracting EDF parameters from /content/data/Subject31_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 39999  =      0.000 ...    79.998 secs...
Extracting EDF parameters from /content/data/Subject01_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Extracting EDF parameters from /content/data/Subject00_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    61.998 secs...
Extracting EDF parameters from /content/data/Subject16_1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 90999  =      0.000 ...   181.998 secs...
Extracting EDF parameters from /content/data/Subject16_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30999  =      0.000 ...    

In [5]:
data1[0].shape

(18, 40000)

In [0]:
dim=500
def reshape(data):   
    shape=data.ravel().shape[0]
    s=shape%(18*500)
    if s!=0:
        d=data.ravel()[s//2:-s//2].reshape(-1,dim,18)
        #return np.swapaxes(d,1,2)   
        return d
    else: 
        d1=data.reshape(-1,dim,18)
        #return np.swapaxes(d1,1,2)   
        return d1

In [0]:
data_1=[]
for i in data1:
    data_1.append(reshape(i))
    
    
data_2=[]
for i in data2:
    data_2.append(reshape(i))       

In [8]:
len(data_1),len(data_2)

(36, 36)

In [9]:
import keras 
from tensorflow.keras.layers import Conv1D,MaxPool1D,AveragePooling1D,GlobalAveragePooling1D,Dropout,Dense,BatchNormalization,Flatten,LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam,RMSprop,SGD
#from tensorflow.keras.activations import ReLU
from tensorflow.keras.layers import LeakyReLU
from sklearn.model_selection import StratifiedKFold    
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from sklearn.utils import shuffle
import tensorflow

Using TensorFlow backend.


In [12]:

history=[]
test_accuracy=[]
val_accuracy=[]
for i in range(35):
    HC=data_1.copy()
    SZ=data_2.copy()
    #train, test, val data
    Xtest0=np.array(HC.pop(i))
    Xval0=np.array(HC.pop(i+1))
    Xtrain0=np.concatenate(HC)
    #create labels for them
    ytest0=np.zeros(Xtest0.shape[0])
    yval0=np.zeros(Xval0.shape[0])
    ytrain0=np.zeros(Xtrain0.shape[0])
    
    #create rain, test, val data
    Xtest1=np.array(SZ.pop(i))
    Xval1=np.array(SZ.pop(i+1))
    Xtrain1=np.concatenate(SZ)

    ytest1=np.ones(Xtest1.shape[0])
    yval1=np.ones(Xval1.shape[0])
    ytrain1=np.ones(Xtrain1.shape[0])
    #combined data
    X_test=np.concatenate((Xtest0,Xtest1))
    X_val=np.concatenate((Xval0,Xval1))
    X_train=np.concatenate((Xtrain0,Xtrain1))
    #combined labesl
    y_test=np.concatenate((ytest0,ytest1))
    y_val=np.concatenate((yval0,yval1))
    y_train=np.concatenate((ytrain0,ytrain1))
    
    


    # y_val=tensorflow.keras.utils.to_categorical(y_val)
    # y_test=tensorflow.keras.utils.to_categorical(y_test)
    # y_train=tensorflow.keras.utils.to_categorical(y_train)
    
    scaler=StandardScaler()
    scalers = {}
    for i in range(X_train.shape[2]):
        scalers[i] = StandardScaler()
        X_train[:, :, i] = scalers[i].fit_transform(X_train[:, :, i]) 
    
    for i in range(X_test.shape[2]):
        X_test[:, :, i] = scalers[i].transform(X_test[:, :, i])
    
    for i in range(X_test.shape[2]):
        X_val[:, :, i] = scalers[i].transform(X_val[:, :, i])
        
    X_train,y_train=shuffle(X_train,y_train)
    #X_val,y_val=shuffle(X_val,y_val)
    #X_test,y_test=shuffle(X_test,y_test)

    
    model=Sequential()
    model.add(LSTM(64, activation= 'tanh',recurrent_activation ='sigmoid',input_shape=(500,18),recurrent_dropout=0,unroll =False,use_bias =True,return_sequences = True))
    model.add(Conv1D(filters=32,kernel_size=3,strides=1,activation='relu'))#1
    # model.add(BatchNormalization())
    # model.add(LeakyReLU())
    # model.add(MaxPool1D(pool_size=2,strides=2))#2
    model.add(Conv1D(filters=5,kernel_size=3,strides=1,activation='relu'))#3
    # model.add(LeakyReLU())
    # model.add(MaxPool1D(pool_size=2,strides=2))#4
    # model.add(Dropout(0.5))
    # model.add(Conv1D(filters=5,kernel_size=3,strides=1))#5
    # model.add(LeakyReLU())
    # model.add(AveragePooling1D(pool_size=2,strides=2))#6
    # model.add(Dropout(0.5))
    # model.add(Conv1D(filters=5,kernel_size=3,strides=1))#7
    # model.add(LeakyReLU())
    # model.add(AveragePooling1D(pool_size=2,strides=2))#8
    # model.add(Conv1D(filters=5,kernel_size=3,strides=1))#9
    # model.add(LeakyReLU())
    model.add(Flatten())#10
    model.add(Dense(1,activation='sigmoid'))#11
    #model.add(LeakyReLU())

    model.compile(optimizer=Adam(0.0001), loss='binary_crossentropy', metrics=['accuracy'] )
    
    history=model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=30,verbose=2)
    val_accuracy.append(model.evaluate(X_val,y_val)[1])
    test_accuracy.append(model.evaluate(X_test,y_test)[1])

    tensorflow.keras.backend.clear_session()


Epoch 1/30
260/260 - 6s - loss: 0.5802 - accuracy: 0.7299 - val_loss: 0.5773 - val_accuracy: 0.7459
Epoch 2/30
260/260 - 6s - loss: 0.5524 - accuracy: 0.7457 - val_loss: 0.5745 - val_accuracy: 0.7459
Epoch 3/30
260/260 - 6s - loss: 0.5493 - accuracy: 0.7457 - val_loss: 0.5722 - val_accuracy: 0.7459
Epoch 4/30
260/260 - 6s - loss: 0.5468 - accuracy: 0.7457 - val_loss: 0.5647 - val_accuracy: 0.7459
Epoch 5/30
260/260 - 6s - loss: 0.5432 - accuracy: 0.7457 - val_loss: 0.5601 - val_accuracy: 0.7459
Epoch 6/30
260/260 - 6s - loss: 0.5387 - accuracy: 0.7457 - val_loss: 0.5577 - val_accuracy: 0.7459
Epoch 7/30
260/260 - 6s - loss: 0.5357 - accuracy: 0.7462 - val_loss: 0.5460 - val_accuracy: 0.7459
Epoch 8/30
260/260 - 6s - loss: 0.5316 - accuracy: 0.7473 - val_loss: 0.5500 - val_accuracy: 0.7377
Epoch 9/30
260/260 - 6s - loss: 0.5217 - accuracy: 0.7498 - val_loss: 0.5368 - val_accuracy: 0.7377
Epoch 10/30
260/260 - 6s - loss: 0.5148 - accuracy: 0.7548 - val_loss: 0.5222 - val_accuracy: 0.7459

IndexError: ignored

In [11]:
import gc
del model
tensorflow.keras.backend.clear_session()
gc.collect()


1731

In [14]:
np.array(test_accuracy).mean()

0.7153047656311708

In [13]:
np.array(val_accuracy).mean()

0.7172612144666559